In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
# -------------------------------
# 1) LOAD FEATURES AND LABELS
# -------------------------------
data = np.load("efficientnet_b0_features_pt.npz")  # or .npz file from Keras code if needed
X = data["features"]  # Shape: (num_samples, 1280) for EfficientNet-B0
y = data["labels"]    # Shape: (num_samples,)

# Convert to PyTorch tensors
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).long()  # Assuming multi-class integer labels

In [3]:
# ---------------------------------------------------
# 2) SPLIT DATA: 75% TRAIN, 10% VAL, 15% TEST
# ---------------------------------------------------
# First split: train vs. (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_tensor,
    y_tensor,
    train_size=0.75,
    shuffle=True,
    stratify=y_tensor,
    random_state=42
)

# Among the remaining 25%, we want 10% val, 15% test (ratio 2:3).
# That means test is 0.6 (3/5) of the temp set, val is 0.4 (2/5).
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.6,
    shuffle=True,
    stratify=y_temp,
    random_state=42
)

print(f"Train size: {len(X_train)}, Val size: {len(X_val)}, Test size: {len(X_test)}")

Train size: 48563, Val size: 6475, Test size: 9713


In [4]:
# ---------------------------------------------------
# 3) CREATE DATASET + DATALOADER CLASSES
# ---------------------------------------------------
class FeatureDataset(Dataset):
    """
    A simple Dataset that wraps feature vectors and labels.
    """
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]
        return x, y

# Create Dataset instances
train_dataset = FeatureDataset(X_train, y_train)
val_dataset   = FeatureDataset(X_val, y_val)
test_dataset  = FeatureDataset(X_test, y_test)

# Create DataLoaders (use mini-batches for training)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

In [52]:
# ---------------------------------------------------
# 4) DEFINE A SIMPLE FEEDFORWARD NETWORK
# ---------------------------------------------------
import torch
import torch.nn as nn

class DeeperNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(DeeperNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()

        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()

        self.fc3 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.relu3 = nn.ReLU()

        self.fc4 = nn.Linear(hidden_dim // 2, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        x = self.relu3(x)

        x = self.fc4(x)
        return x

num_features = X_train.shape[1]  # 1280 if using EfficientNet-B0
num_classes  = len(torch.unique(y_train))

model = DeeperNet(input_dim=num_features, hidden_dim=256, num_classes=num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function (CrossEntropy for classification)
criterion = nn.CrossEntropyLoss()

# Define optimizer (Adam, for example)
optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=1e-5)

In [53]:
import copy

epochs = 200  # Adjust as needed
patience = 5  # Number of epochs to wait after last improvement in val loss
best_val_loss = float('inf')
patience_counter = 0

# Keep a copy of the best model weights
best_model_weights = copy.deepcopy(model.state_dict())

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    # --------------------
    # Training step
    # --------------------
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward + Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Compute average train loss
    avg_train_loss = running_loss / len(train_loader)

    # --------------------
    # Validation step
    # --------------------
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    # Print progress
    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}")

    # --------------------
    # Early Stopping Check
    # --------------------
    if avg_val_loss < best_val_loss:
        # Found an improvement, save the model weights
        best_val_loss = avg_val_loss
        best_model_weights = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        # No improvement, increase counter
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}.")
            break

# Restore the best weights
model.load_state_dict(best_model_weights)
print(f"Training finished. Best validation loss: {best_val_loss:.4f}")


Epoch [1/200] Train Loss: 2.5636, Val Loss: 2.5616
Epoch [2/200] Train Loss: 2.5599, Val Loss: 2.5575
Epoch [3/200] Train Loss: 2.5551, Val Loss: 2.5519
Epoch [4/200] Train Loss: 2.5481, Val Loss: 2.5431
Epoch [5/200] Train Loss: 2.5363, Val Loss: 2.5274
Epoch [6/200] Train Loss: 2.5140, Val Loss: 2.4961
Epoch [7/200] Train Loss: 2.4664, Val Loss: 2.4265
Epoch [8/200] Train Loss: 2.3642, Val Loss: 2.2904
Epoch [9/200] Train Loss: 2.2150, Val Loss: 2.1429
Epoch [10/200] Train Loss: 2.0815, Val Loss: 2.0271
Epoch [11/200] Train Loss: 1.9809, Val Loss: 1.9436
Epoch [12/200] Train Loss: 1.9080, Val Loss: 1.8834
Epoch [13/200] Train Loss: 1.8512, Val Loss: 1.8332
Epoch [14/200] Train Loss: 1.8024, Val Loss: 1.7904
Epoch [15/200] Train Loss: 1.7596, Val Loss: 1.7483
Epoch [16/200] Train Loss: 1.7214, Val Loss: 1.7136
Epoch [17/200] Train Loss: 1.6882, Val Loss: 1.6829
Epoch [18/200] Train Loss: 1.6593, Val Loss: 1.6597
Epoch [19/200] Train Loss: 1.6336, Val Loss: 1.6352
Epoch [20/200] Train 

In [1]:
# ---------------------------------------------------
# 6) EVALUATION ON TEST SET (METRICS)
# ---------------------------------------------------
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        # For classification, the predicted class is argmax
        _, predicted = torch.max(outputs, dim=1)

        all_preds.append(predicted.cpu().numpy())
        all_labels.append(batch_y.cpu().numpy())

# Flatten lists
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Accuracy, Precision, Recall (multi-class setting)
acc = accuracy_score(all_labels, all_preds)
# "weighted" handles label imbalance for multi-class
prec = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
rec = recall_score(all_labels, all_preds, average='weighted', zero_division=0)

print("=== TEST METRICS ===")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {prec:.4f}")
print(f"Recall:     {rec:.4f}")

NameError: name 'model' is not defined